In [1]:
import os
import pandas as pd

os.chdir("C:\\Users\\ndgig\\Repositories\\grammatron")
print(os.getcwd())

%load_ext autoreload
%autoreload 2

from grammatron.grammatron import Grammatron

C:\Users\ndgig\Repositories\grammatron


In [2]:
splits = {
    "train": "train_consumer_complaints.csv",
    "test": "test_consumer_complaints.csv",
}
df = pd.read_csv(
    "hf://datasets/Johnade/consumer_complaints_cfpb/" + splits["test"]
).sample(1000, random_state=32)

docs = df["consumer_complaint_narrative"].to_list()

docs[:5]

['Navient is my student loan provider. I also owe from a student loan from a  XXXX  college, the loan borrower is Nelnet. The Nelnet lo an is from  XXXX , this is what I owed then {$3200.00}. I was paying on it for a long time then I started going back to school in  XXXX  to get a  XXXX   XXXX  so I deferred. I still owe {$4000.0 0} on the Nelnet loan, I asked Nelent to  forgive my loan and they denied also. Since I have had  XXXX , they have emailed me about deferring every time I get behind so I defer. What was a  XXXX  loan is now close t o  XXXX . This is because of deferring payments and interest. Today I am behind three months. The beginning of the year I asked for a  XXXX  forbearance for the school year and was denied. Today I looked into def  erring and if I defer again the balance of my loan will be close t  o 70,000. How am I ever to get this paid in my lifetime when the interest is outrageous?',
 'In accordance with the Fair Credit Reporting Act. The Account below violated 

In [3]:
if "ng" in locals():
    del ng
ng = Grammatron("sentence-transformers/all-mpnet-base-v2", device="cuda", invalid_start_token_pattern=r"^##")
ng

In [4]:
# inputs, token_embeds, attns = ng.encode(docs, batch_size=512, amp=False)
# token_embeds.shape, attns.shape

In [5]:
# attns

In [6]:
# i = 0
# tokens = [ng.tokenizer.decode(x) for x in inputs["input_ids"][i]]
# token_attns = attns[i]
# pd.Series(token_attns, index=tokens).sort_values(ascending=False).head(25)

In [7]:
import torch

In [8]:
samp_idx, ngrams, ngram_vecs = ng.encode_extract(docs, batch_size=512, ngram_range=(4, 6), amp=True, amp_dtype=torch.bfloat16)
ngram_vecs

Encoding:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting:   0%|          | 0/3 [00:00<?, ?it/s]

array([[ 1.03039041e-01,  1.05794705e-01, -4.34091426e-02, ...,
         8.29020366e-02, -1.13730624e-01, -8.91451538e-02],
       [ 1.28737420e-01,  5.14399856e-02, -1.04864217e-01, ...,
         7.68212229e-02, -1.04046993e-01, -4.94596809e-02],
       [ 1.33087844e-01,  1.32640123e-01, -1.17653444e-01, ...,
         9.16607678e-02, -1.16081834e-01, -8.16160142e-02],
       ...,
       [ 1.66979551e-01,  7.50724673e-02,  8.52259994e-03, ...,
         7.56566003e-02, -2.12111160e-01,  3.21244188e-05],
       [ 1.83934569e-01,  1.35386586e-01,  2.11149678e-02, ...,
         6.76523149e-02, -2.00338140e-01,  9.05787852e-03],
       [ 1.61530018e-01,  1.72564283e-01,  2.39087660e-02, ...,
         5.22188544e-02, -1.88333020e-01,  2.63039619e-02]], dtype=float32)

In [17]:
ng.tokenizer.get_vocab()

{'decorative': 11588,
 'sodium': 13369,
 'employing': 15444,
 'mr': 2724,
 '83': 6644,
 '=': 1031,
 'boo': 22021,
 'gettysburg': 22581,
 'lineage': 13325,
 'sainte': 16951,
 'preparation': 7551,
 'ე': 1446,
 '##hism': 23112,
 '1800s': 19882,
 'clawed': 22548,
 'their': 2041,
 'casey': 9040,
 '广': 1846,
 'coefficients': 21378,
 '##table': 10884,
 'baskets': 25950,
 'hounds': 27776,
 '##ии': 15418,
 '##rak': 16559,
 'senior': 3030,
 'glare': 10986,
 'oath': 11296,
 'tito': 20716,
 '##hips': 19805,
 'executions': 22683,
 '##minster': 24435,
 '##phine': 20742,
 'substrate': 16309,
 'legacy': 8031,
 'interpreting': 25459,
 'rubs': 28864,
 '338': 27912,
 'tanzania': 11963,
 'elemental': 19533,
 'plots': 14815,
 '[unused730]': 739,
 'formation': 4199,
 '##rys': 24773,
 'arterial': 25547,
 '[unused949]': 958,
 'southwest': 4947,
 '[unused310]': 319,
 'exasperated': 24383,
 'concrete': 5513,
 'kerry': 11264,
 'blaming': 24118,
 '##hesion': 21475,
 'scowled': 17478,
 'ⱼ': 1635,
 'crab': 18085,
 

In [18]:
ng.tokenizer.convert_ids_to_tokens(ng.invalid_start_token_ids)

['##hism',
 '##table',
 '##ии',
 '##rak',
 '##hips',
 '##minster',
 '##phine',
 '##rys',
 '##hesion',
 '##ulating',
 '##balance',
 '##gonal',
 '##ses',
 '##糹',
 '##´',
 '##aging',
 '##dt',
 '##ling',
 '##ographer',
 '##ན',
 '##by',
 '##physical',
 '##bson',
 '##wart',
 '##そ',
 '##oides',
 '##nar',
 '##°',
 '##urance',
 '##hole',
 '##lad',
 '##acle',
 '##ity',
 '##wana',
 '##onic',
 '##ø',
 '##redo',
 '##⁶',
 '##iface',
 '##ɡ',
 '##bbled',
 '##dlow',
 '##pressed',
 '##rz',
 '##un',
 '##lizer',
 '##ement',
 '##grounds',
 '##yen',
 '##│',
 '##gne',
 '##mt',
 '##ntino',
 '##lance',
 '##door',
 '##lz',
 '##ła',
 '##ock',
 '##nko',
 '##nal',
 '##stan',
 '##ヒ',
 '##fication',
 '##ex',
 '##ona',
 '##orro',
 '##usions',
 '##漢',
 '##clusive',
 '##fell',
 '##sc',
 '##劉',
 '##cho',
 '##sey',
 '##dridge',
 '##≈',
 '##actic',
 '##oshi',
 '##awan',
 '##cave',
 '##saka',
 '##hima',
 '##phile',
 '##nem',
 '##gold',
 '##lvis',
 '##ifying',
 '##内',
 '##osaurus',
 '##ᴵ',
 '##ₛ',
 '##tones',
 '##ち',
 '##ra

In [19]:
ngrams[samp_idx == 2]

array(['i have been making', 'have been making payments',
       'been making payments on', 'making payments on my',
       'payments on my student', 'on my student loan',
       'my student loan and', 'student loan and it', 'loan and it has',
       'and it has been', 'it has been reported', 'has been reported i',
       "been reported i '", "reported i'm", "i'm 3", "' m 3 months",
       'm 3 months behind', '3 months behind i', 'months behind i haven',
       "behind i haven '", "i haven't", "haven't made", "' t made mid",
       't made mid payments', 'made mid payments at',
       'mid payments at all', 'payments at all for', 'at all for the',
       'all for the last', 'for the last 4', 'the last 4yr', 'last 4yrs',
       '4yrs now', 'now they want to', 'they want to find',
       'want to find me', 'to find me in', 'find me in default',
       'i have been making payments', 'have been making payments on',
       'been making payments on my', 'making payments on my student',
    

In [20]:
print(docs[0])

Navient is my student loan provider. I also owe from a student loan from a  XXXX  college, the loan borrower is Nelnet. The Nelnet lo an is from  XXXX , this is what I owed then {$3200.00}. I was paying on it for a long time then I started going back to school in  XXXX  to get a  XXXX   XXXX  so I deferred. I still owe {$4000.0 0} on the Nelnet loan, I asked Nelent to  forgive my loan and they denied also. Since I have had  XXXX , they have emailed me about deferring every time I get behind so I defer. What was a  XXXX  loan is now close t o  XXXX . This is because of deferring payments and interest. Today I am behind three months. The beginning of the year I asked for a  XXXX  forbearance for the school year and was denied. Today I looked into def  erring and if I defer again the balance of my loan will be close t  o 70,000. How am I ever to get this paid in my lifetime when the interest is outrageous?


In [21]:
ngrams[samp_idx == 0], ngram_vecs[samp_idx == 0]

(array(['navient is', 'is my student loan', 'my student loan provider',
        'student loan provider.', 'loan provider. i', 'provider. i also',
        '. i also owe', 'i also owe from', 'also owe from a',
        'owe from a student', 'from a student loan', 'a student loan from',
        'student loan from a', 'loan from a xx', 'from a xxxx',
        'a xxxx college', 'xxxx college,', 'college, the loan',
        ', the loan borrow', 'the loan borrower', 'loan borrower is',
        'borrower is ne', 'is nelnet', 'nelnet.', '. the neln',
        'the nelnet', 'nelnet lo', 'lo an is from', 'an is from xx',
        'is from xxxx', 'from xxxx,', 'xxxx, this', ', this is what',
        'this is what i', 'is what i owed', 'what i owed then',
        'i owed then {', 'owed then { $', 'then { $ 320', '{ $ 3200',
        '$ 3200.', '3200. 00', '. 00 }.', '00 }. i', '}. i was',
        '. i was paying', 'i was paying on', 'was paying on it',
        'paying on it for', 'on it for a', 'it for 

In [22]:
queries = ng.encode_queries(["interest rate is too high", "fraud"], amp=True, amp_dtype=torch.bfloat16)
queries

Encoding:   0%|          | 0/1 [00:00<?, ?it/s]

Extracting:   0%|          | 0/2 [00:00<?, ?it/s]

array([[ 0.03413197, -0.06059905, -0.04677777, ...,  0.01644105,
        -0.04462195,  0.01331554],
       [ 0.07211701,  0.30349678, -0.12031689, ...,  0.10713675,
        -0.13139658,  0.05263014]], dtype=float32)

In [29]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=15, radius=0.5, metric="cosine")
nn.fit(ngram_vecs)
dists, idx = nn.radius_neighbors(queries)
ngrams[idx[0]], samp_idx[idx[0]]

(array(['when the interest is', 'the interest is outrageous',
        'interest is outrageous?', '00 } in interest', '} in interesttha',
        '. 00 } in', '00 } in interest', '} in interesttha',
        'in interestthat', 'interest of xxxx', 'interest would begin to',
        'time my interest is', 'would earn interest rates',
        'earn interest rates that', 'interest rates that are',
        'to charge more interest', 'charge more interest which',
        'more interest which is', 'to reduce interest rates',
        'reduce interest rates could', 'interest rates could just',
        'and the interest rate', 'the interest rate was',
        'interest rate was supposed', 'rate was supposed to',
        'approximately 19 % interest', '19 % interest for',
        '% interest for over', 'interest for over $',
        'want my interest rate', 'my interest rate reduced',
        'interest rate reduced.', ') raised my interest',
        'raised my interest rate', 'my interest rate near

In [16]:
[ngrams[0][i] for i in idx[0]]

IndexError: string index out of range

In [ ]:
ngrams[0]

['john was walking',
 'was walking down',
 'walking down the',
 'down the street',
 'the street when',
 'street when he',
 'when he saw',
 'he saw a',
 'saw a vendor',
 'a vendor.',
 'vendor. he',
 '. he was',
 'he was so',
 'was so excited',
 'so excited!',
 'excited! he',
 '! he wanted',
 'he wanted to',
 'wanted to buy',
 'to buy something',
 'buy something,',
 'something, but',
 ', but his',
 'but his mommy',
 'his mommy said',
 'mommy said he',
 'said he had',
 'he had to',
 'had to wait',
 'to wait.',
 'wait. he',
 '. he watched',
 'he watched the',
 'watched the vendor',
 'the vendor with',
 'vendor with big',
 'with big eyes',
 'big eyes,',
 'eyes, but',
 ', but the',
 'but the vendor',
 'the vendor started',
 'vendor started to',
 'started to ignore',
 'to ignore him',
 'ignore him.',
 'him. john',
 '. john started',
 'john started to',
 'started to feel',
 'to feel sad',
 'feel sad until',
 'sad until he',
 'until he saw',
 'he saw the',
 'saw the modern',
 'the modern vendor